In [1]:
import numpy as np
import helper_functions as hf
from scipy.stats import kurtosis, skew
from biosppy.signals.emg import emg
from biosppy.signals.eeg import eeg
from neurokit import emg_process


In [2]:
X_train_eeg1, _ = hf.read_csv_to_matrix("task5/input/train_eeg1.csv", "Id")
X_train_eeg2, _ = hf.read_csv_to_matrix("task5/input/train_eeg2.csv", "Id")
X_train_emg, _ = hf.read_csv_to_matrix("task5/input/train_emg.csv", "Id")
X_test_eeg1, _ = hf.read_csv_to_matrix("task5/input/test_eeg1.csv", "Id")
X_test_eeg2, _ = hf.read_csv_to_matrix("task5/input/test_eeg2.csv", "Id")
X_test_emg, _ = hf.read_csv_to_matrix("task5/input/test_emg.csv", "Id")
y_train, _ = hf.read_csv_to_matrix("task5/input/train_labels.csv", "Id")
_, test_index = hf.read_csv_to_matrix("task5/input/sample.csv", "Id")
y_train = np.squeeze(y_train)


In [3]:
def signal_feature_extraction(x):
    x_new = []
    for i in range(x.shape[0]):
        x_new.append([np.mean(x[i]), np.std(x[i]), kurtosis(x[i]), skew(x[i])])
    return np.asarray(x_new)


def eeg_feature_extraction(x):
    return eeg(signal=x, sampling_rate=128, show=False)


def emg_feature_extraction(x):
    # dwt
    return


In [59]:
a = eeg(X_train_eeg1[0].reshape(-1, 512).transpose(), sampling_rate=128, show=False)


In [67]:
a['theta'].transpose()

array([[1.33670502e-12, 5.06019398e-12, 1.14840663e-11, 1.14840663e-11,
        4.76238258e-11, 4.76238258e-11, 4.76238258e-11, 8.49887724e-11,
        5.83229822e-11, 3.45222628e-11, 3.19295779e-11, 5.83229822e-11,
        3.19295779e-11, 2.67963386e-11, 2.67963386e-11, 5.62898487e-11,
        5.62898487e-11, 5.62965543e-11, 5.62965543e-11, 5.62965543e-11,
        4.32487175e-11, 4.32487175e-11, 4.32487175e-11, 4.32487175e-11,
        3.64272151e-11, 4.03346200e-11, 3.00235102e-11, 3.00235102e-11,
        3.35738951e-11, 3.35738951e-11, 3.00235102e-11]])

In [54]:
import pywt
import matplotlib.pyplot as plt
import numpy as np

ts = X_train_emg[0]

(ca, cd) = pywt.dwt(ts, 'haar')

cat = pywt._thresholding.soft(ca, np.std(ca)/2)
cdt = pywt._thresholding.soft(cd, np.std(cd)/2)

ts_rec = pywt.idwt(cat, cdt, 'haar')

plt.close('all')

plt.subplot(211)
# Original coefficients
plt.plot(ca, '--*b')
plt.plot(cd, '--*r')
# Thresholded coefficients
plt.plot(cat, '--*c')
plt.plot(cdt, '--*m')
plt.legend(['ca','cd','ca_thresh', 'cd_thresh'], loc=0)
plt.grid('on')

plt.subplot(212)
plt.plot(ts)
#plt.hold('on')
plt.plot(ts_rec, 'r')
plt.legend(['original signal', 'reconstructed signal'])
plt.grid('on')
plt.show()